### Face Detection system using yolov8n

In [1]:
## moving all .xml file

In [2]:
import os
import shutil

source_directory = "train"
destination_directory = "./label"

os.makedirs(destination_directory, exist_ok=True)

for filename in os.listdir(source_directory):
    if filename.endswith(".xml"):
        source_path = os.path.join(source_directory, filename)
        destination_path = os.path.join(destination_directory, filename)
        shutil.move(source_path, destination_path)

print("All .xml files have been moved to the destination folder.")


All .xml files have been moved to the destination folder.


### Changing classes

In [3]:
import os
import xml.etree.ElementTree as ET

xml_directory = 'label'

def construct_standard_name(roll_number):
    return f"fa-22-{roll_number}-G"  # Adjust 'fa-22-' and '-G' as necessary

# Loop through all XML files in the specified directory
for filename in os.listdir(xml_directory):
    if filename.endswith('.xml'):  # Assuming label files are in .xml format
        file_path = os.path.join(xml_directory, filename)
        
        # Parse the XML file
        tree = ET.parse(file_path)
        root = tree.getroot()

        # Update names
        for name in root.findall('.//name'):
            current_name = name.text.strip()
            # Check if the current name contains just a roll number or already in the desired format
            if len(current_name.split('-')) == 1:  # Only a roll number
                roll_number = current_name
                standardized_name = construct_standard_name(roll_number)
                name.text = standardized_name
            elif '-' in current_name:  # If it has the format 'fa-22-294-G' or similar
                parts = current_name.split('-')
                # Assuming the roll number is always the third part
                if len(parts) > 2:
                    roll_number = parts[2]
                    standardized_name = construct_standard_name(roll_number)
                    name.text = standardized_name
        
        # Write the updated XML back to the file
        tree.write(file_path)

print("XML name format conversion completed.")


XML name format conversion completed.


In [4]:
import os
import xml.etree.ElementTree as ET

xml_folder = 'label/'     
label_folder = 'labels/'          

os.makedirs(label_folder, exist_ok=True)

class_name_to_id = {}

def convert_bbox(size, box):
    """
    Convert bounding box to YOLO format.
    `size` is the size of the image (width, height).
    `box` is the bounding box (xmin, xmax, ymin, ymax).
    """
    dw = 1.0 / size[0]
    dh = 1.0 / size[1]
    x = (box[0] + box[1]) / 2.0
    y = (box[2] + box[3]) / 2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    return x * dw, y * dh, w * dw, h * dh

def convert_annotation(xml_file, label_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)
    
    with open(label_file, 'w') as f:
        for obj in root.iter('object'):
            cls_name = obj.find('name').text
            if cls_name not in class_name_to_id:
                # Assign a new class ID if it's not already in the dictionary
                class_name_to_id[cls_name] = len(class_name_to_id)
            cls_id = class_name_to_id[cls_name]
            
            # Get bounding box coordinates
            xml_box = obj.find('bndbox')
            b = (
                int(xml_box.find('xmin').text), 
                int(xml_box.find('xmax').text),
                int(xml_box.find('ymin').text), 
                int(xml_box.find('ymax').text)
            )
            bbox = convert_bbox((w, h), b)
            f.write(f"{cls_id} {' '.join(map(str, bbox))}\n")

# Process each XML file
for xml_file in os.listdir(xml_folder):
    if not xml_file.endswith('.xml'):
        continue
    xml_path = os.path.join(xml_folder, xml_file)
    label_path = os.path.join(label_folder, xml_file.replace('.xml', '.txt'))
    convert_annotation(xml_path, label_path)

# Save the class_name_to_id mapping to a file for reference
with open('classes.txt', 'w') as f:
    for class_name, class_id in class_name_to_id.items():
        f.write(f"{class_id}: {class_name}\n")
        print(f"{class_id}: {class_name}")

print(f"\nTotal number of classes: {len(class_name_to_id)}")


print("Conversion completed! Check the 'labels/' folder for YOLO format labels.",len(class_name_to_id))


0: fa-22-objects-G
1: fa-22-abdullah azmat-G
2: fa-22-watch-G

Total number of classes: 3
Conversion completed! Check the 'labels/' folder for YOLO format labels. 3


In [5]:
import os
import xml.etree.ElementTree as ET

# Define the directory where your XML files are stored
xml_directory = './label'

# The incorrect and correct roll numbers
incorrect_rollnumber = "fa-22-106-G"
correct_rollnumber = "fa-22-108-G"

# Loop through all XML files in the specified directory
for filename in os.listdir(xml_directory):
    if filename.endswith('.xml'):  # Assuming label files are in .xml format
        file_path = os.path.join(xml_directory, filename)
        
        # Parse the XML file
        tree = ET.parse(file_path)
        root = tree.getroot()

        # Update names
        for name in root.findall('.//name'):
            current_name = name.text.strip()
            # Check if the current name is the incorrect one
            if current_name == incorrect_rollnumber:
                name.text = correct_rollnumber  # Update to correct roll number
        
        # Write the updated XML back to the file
        tree.write(file_path)

print("Roll number correction completed.")


Roll number correction completed.


In [6]:
# Download ing dataset
url = 'https://drive.google.com/file/d/19-JTxbAuOl22abV4dvINvciLeUTpwlyi/view?usp=drive_link'
import requests
response = requests.get(url)
with open('data.zip', 'wb') as f:
  f.write(response.content)

ConnectionError: HTTPSConnectionPool(host='drive.google.com', port=443): Max retries exceeded with url: /file/d/19-JTxbAuOl22abV4dvINvciLeUTpwlyi/view?usp=drive_link (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001F0A419B770>: Failed to resolve 'drive.google.com' ([Errno 11001] getaddrinfo failed)"))

In [ ]:
import zipfile
with zipfile.ZipFile('https://drive.google.com/file/d/19-JTxbAuOl22abV4dvINvciLeUTpwlyi/view?usp=drive_link') as zip_ref:
    zip_ref.extractall()

OSError: [Errno 22] Invalid argument: 'https://drive.google.com/file/d/19-JTxbAuOl22abV4dvINvciLeUTpwlyi/view?usp=drive_link'

In [ ]:
!pip install ultralytics
from ultralytics import YOLO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.0/879.0 kB 53.8 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
import os
os.chdir('/content/dataset')

In [ ]:
ls

data.yaml  test/  train/  val/


In [ ]:
# Disable W&B
import os
os.environ['WANDB_MODE'] = 'disabled'
!yolo task=detect mode=train epochs=100 batch=32 plots=True model='yolov8n.pt' data=data.yaml imgsz=640

Ultralytics 8.3.27 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=100, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_wid

In [ ]:
import shutil


source_folder = '/content/datasets/runs'
destination_folder = '/content/drive/MyDrive/face attandance system/'

# Move the folder
shutil.move(source_folder, destination_folder)

print("Folder moved successfully!")


Folder moved successfully!


In [ ]:
!pip install --upgrade onnxruntime-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.5/291.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.2 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO

model = YOLO("/content/drive/MyDrive/face attandance system/runs/detect/train2/weights/best.pt")

model.export(format="onnx")  # This creates 'best.onnx'


Ultralytics 8.3.27 🚀 Python-3.10.12 torch-2.5.0+cu121 CPU (Intel Xeon 2.00GHz)
Model summary (fused): 168 layers, 3,010,913 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/content/drive/MyDrive/face attandance system/runs/detect/train2/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 31, 8400) (6.0 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim', 'onnxruntime'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 182.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 89.2 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 7.9s, installed 3 packages: ['onnx>=1.12.0', 'onnxslim', 'onnxruntime']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.36...
ONNX: export su

'/content/drive/MyDrive/face attandance system/runs/detect/train2/weights/best.onnx'

In [ ]:
import os
os.chdir('../content/drive/MyDrive/face attandance system/runs/detect/train2/weights')

In [ ]:
os.chdir('./drive/MyDrive/face attandance system/runs/detect/train2/weights')

FileNotFoundError: [Errno 2] No such file or directory: 'drive/MyDrive/face attandance system/runs/detect/train2/weights'

In [ ]:
!yolo task=predict batch=32 plots=True model='yolov8n.pt' data=data.yaml imgsz=640